In [83]:
from pdfminer.converter import TextConverter, PDFPageAggregator
from pdfminer.layout import LAParams, LTTextBoxHorizontal, LTRect, LTLine
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
import time

In [92]:
import os
file_names = []
for path, subdir, files in os.walk('./PDF_data'):
    for name in files:
        file_names.append(os.path.join(path, name))

In [540]:
data = []
failed = []

In [544]:
start = 0

In [1]:
count = start
other_count = 0
for file in range(count, len(file_names)):
    #Open the PDF File as an object
    if 'crdownload' in file_names[file]:
        continue
    pdfFileObj = open(file_names[file], 'rb')
    print(file_names[file])

    #Create a parser for the pdf object
    parser = PDFParser(pdfFileObj)

    #Create a resource manager
    rsrcmgr = PDFResourceManager()

    # Set parameters for analysis.
    laparams = LAParams(char_margin=50, word_margin=50)

    # Create a PDF page aggregator object.
    device = PDFPageAggregator(rsrcmgr, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    
    temp = {}
    temp['file_name'] = file
    principal = 0
    street = 0
    type_flag = 0
    name_flag = 0
    ma_flag = 0
    agent_flag = 0
    principal_flag = 0
    exec_flag = 0
    lock_flag = 0
    end_count = ''
    try:
        for page in PDFPage.get_pages(pdfFileObj):
            interpreter.process_page(page)
            layout = device.get_result()
            for index,element in enumerate(layout):
                try:
                    t = element.get_text()
                except:
                    continue
                if type_flag == 1: ## To get multiple lines of business type description
                    temp['Type'] = t.strip(' \n')
                    type_flag = 0
                if exec_flag == 1: ## To get multiple lines of executives and their addressses
                    try:
                        get_count = t.split('.', 1)[0]
                        if len(get_count) > 1 and len(get_count) <= 3:
                            get_count = get_count[0]
                        elif len(get_count) > 3:
                            get_count = -1
                        get_count = int(get_count)
                    except:
                        get_count = -1
                    if get_count >= end_count:
                        exec_flag = 0
                    else:
                        if exec_count > 2:
                            temp['Execs'].append(t)
                    exec_count += 1
                if agent_flag == 2: ## To get the resident agent in some special cases
                    try:
                        temp['Agent'] = t.split('Name:')[1]
                    except:
                        pass
                    agent_flag = 0
                if principal_flag == 1: ## To get multiple lines of principal address
                    try:
                        get_count = t.split('.', 1)[0]
                        if len(get_count) > 1 and len(get_count) <= 3:
                            get_count = get_count[0]
                        elif len(get_count) > 3:
                            get_count = -1
                        get_count = int(get_count)
                    except:
                        get_count = -1
                    if get_count >= end_count:
                        principal_flag = 0
                    else:
                        temp['Principal'] += t.strip(' ')
                if ma_flag == 1: ## To get multiple lines of local address
                    try:
                        get_count = t.split('.', 1)[0]
                        if len(get_count) > 1 and len(get_count) <= 3:
                            get_count = get_count[0]
                        elif len(get_count) > 3:
                            get_count = -1
                        get_count = int(get_count)
                    except:
                        get_count = -1
                    if get_count >= end_count:
                        ma_flag = 0
                    else:
                        if agent_flag == 1:
                            agent_flag = 0
                            temp['Agent'] = t.split('\n', 1)[0].strip(' ')
                            temp['MALoc'] += t.split('\n', 1)[1].strip(' ')
                        else:
                            temp['MALoc'] += t.strip(' ')
                            
                            
                if 'Name' not in temp: ## Get the name of the company
                    if "exact name" in t.lower():
                        temp['Name'] = t.split(":")[1].strip(' \n')
                        if temp['Name'] == "":
                            name_flag = 1
                if 'Type' not in temp: ## To get the business type
                    if "character of business" in t.lower():
                        temp['Type'] = t.split(":", 1)[1].strip(' \n')
                    if "business of the corporation" in t.lower():
                        temp['Type'] = t.split(":", 1)[1].strip(' \n')
                        if temp['Type'] == "":
                            type_flag = 1
                if 'MALoc' not in temp: ## For addresses in the commonwealth
                    if "principal office in massachusetts" in t.lower() \
                    or "office in the commonwealth" in t.lower() and "business in the commonwealth" not in t.lower():
                        if "name of the registered agent" in t.lower() or "address of the resident agent" in t.lower()\
                        and 'Agent' not in temp:
                            agent_flag = 1
                        if len(t.split(':')) > 1:
                            if agent_flag == 1:
                                temp['Agent'] = t.split(':', 1)[1].split('Name:')[1].split('\n')[0]
                                temp['MALoc'] = t.split(':', 1)[1].split('Name:')[1].split('\n', 1)[1]
                                agent_flag = 0
                            else:
                                temp['MALoc'] = t.split(':', 1)[1].strip(' \n')
                                if temp['MALoc'] == '':
                                    ma_flag = 1
                                    end_count = t.split('.', 1)[0]
                                    if len(end_count) > 1:
                                        end_count = end_count[0]
                                    end_count = int(end_count) + 1
                        else:
                            ma_flag = 1
                            temp['MALoc'] = ""
                            end_count = t.split('.', 1)[0]
                            if len(end_count) > 1:
                                end_count = end_count[0]
                            end_count = int(end_count) + 1
                if 'Principal' not in temp: ## To get the principal address
                    if 'principal office:' in t.lower() and 'massachusetts' not in t.lower():
                        principal_flag = 1
                        if len(t.split(':')) > 1:
                            temp['Principal'] = t.split(':', 1)[1].strip(' \n')
                        else:
                            temp['Principal'] = ""
                        end_count = t.split('.', 1)[0]
                        if len(end_count) > 1:
                            end_count = end_count[0]
                        end_count = int(end_count)
                if 'resident agent' in t.lower() and 'Agent' not in temp: ## To get the resident agent
                    try:
                        temp['Agent'] = t.split('\n')[1].split('Name:')[1].strip(' \n')
                    except:
                        agent_flag = 2
                if 'Execs' not in temp: ## To get the executives and their addresses
                    if "all the directors" in t.lower() or "street addresses of all officers" in t.lower():
                        exec_flag = 1
                        temp['Execs'] = []
                        exec_count = 0
                        end_count = t.split('.', 1)[0]
                        if len(end_count) > 1:
                            end_count = end_count[0]
                        end_count = int(end_count)
        if len(temp) > 0:
            data.append(temp)
    except:
        failed.append(file_names[file])
    start += 1

In [574]:
import csv
f = open('business_types.csv', 'w')
writer = csv.writer(f)

writer.writerow(['Name', 'Type'])
for i in data:
    if 'Type' in i and 'Name' in i:
        writer.writerow([i['Name'], i['Type']])

f.close()

In [578]:
import csv
f = open('business_people.csv', 'w')
writer = csv.writer(f)

writer.writerow(['Name', 'Agent', 'Execs'])
for i in data:
    if 'Agent' in i and 'Name' in i and 'Execs' in i:
        writer.writerow([i['Name'], i['Agent'], i['Execs']])

f.close()